In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [26]:
# Step 1: Import the dataset
data = pd.read_csv("C:/Users/mattl/OneDrive/Documents/GitHub/DSC550/Loan_Train.csv")
print(data.head())

    Loan_ID Gender Married Dependents     Education Self_Employed  \
0  LP001002   Male      No          0      Graduate            No   
1  LP001003   Male     Yes          1      Graduate            No   
2  LP001005   Male     Yes          0      Graduate           Yes   
3  LP001006   Male     Yes          0  Not Graduate            No   
4  LP001008   Male      No          0      Graduate            No   

   ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0             5849                0.0         NaN             360.0   
1             4583             1508.0       128.0             360.0   
2             3000                0.0        66.0             360.0   
3             2583             2358.0       120.0             360.0   
4             6000                0.0       141.0             360.0   

   Credit_History Property_Area Loan_Status  
0             1.0         Urban           Y  
1             1.0         Rural           N  
2             1.0   

In [27]:
# Step 2: Data Preprocessing
data = data.drop(columns=["Loan_ID"])
data = data.dropna()
print(data.head())

  Gender Married Dependents     Education Self_Employed  ApplicantIncome  \
1   Male     Yes          1      Graduate            No             4583   
2   Male     Yes          0      Graduate           Yes             3000   
3   Male     Yes          0  Not Graduate            No             2583   
4   Male      No          0      Graduate            No             6000   
5   Male     Yes          2      Graduate           Yes             5417   

   CoapplicantIncome  LoanAmount  Loan_Amount_Term  Credit_History  \
1             1508.0       128.0             360.0             1.0   
2                0.0        66.0             360.0             1.0   
3             2358.0       120.0             360.0             1.0   
4                0.0       141.0             360.0             1.0   
5             4196.0       267.0             360.0             1.0   

  Property_Area Loan_Status  
1         Rural           N  
2         Urban           Y  
3         Urban           Y  
4 

In [28]:
# Step 3: Convert categorical features into dummy variables
y = data["Loan_Status"]
data = data.drop(columns=["Loan_Status"])

categorical_cols = data.select_dtypes(include=['object']).columns
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first'), categorical_cols)
    ],
    remainder='passthrough'
)
X = preprocessor.fit_transform(data)
print(pd.DataFrame(X, columns=preprocessor.get_feature_names_out(data.columns)).head())

   cat__Gender_Male  cat__Married_Yes  cat__Dependents_1  cat__Dependents_2  \
0               1.0               1.0                1.0                0.0   
1               1.0               1.0                0.0                0.0   
2               1.0               1.0                0.0                0.0   
3               1.0               0.0                0.0                0.0   
4               1.0               1.0                0.0                1.0   

   cat__Dependents_3+  cat__Education_Not Graduate  cat__Self_Employed_Yes  \
0                 0.0                          0.0                     0.0   
1                 0.0                          0.0                     1.0   
2                 0.0                          1.0                     0.0   
3                 0.0                          0.0                     0.0   
4                 0.0                          0.0                     1.0   

   cat__Property_Area_Semiurban  cat__Property_Area_Urba

In [29]:
# Step 4: Split the data into training and test sets, with "Loan_Status" as the target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
# Step 5: Create a pipeline with Min-Max Scaler and KNN Classifier
pipeline = Pipeline([
    ('scaler', MinMaxScaler()),
    ('model', KNeighborsClassifier())
])

In [32]:
# Step 6: Fit the default KNN classifier to the data
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
accuracy_default = accuracy_score(y_test, y_pred)
print(data.head())

  Gender Married Dependents     Education Self_Employed  ApplicantIncome  \
1   Male     Yes          1      Graduate            No             4583   
2   Male     Yes          0      Graduate           Yes             3000   
3   Male     Yes          0  Not Graduate            No             2583   
4   Male      No          0      Graduate            No             6000   
5   Male     Yes          2      Graduate           Yes             5417   

   CoapplicantIncome  LoanAmount  Loan_Amount_Term  Credit_History  \
1             1508.0       128.0             360.0             1.0   
2                0.0        66.0             360.0             1.0   
3             2358.0       120.0             360.0             1.0   
4                0.0       141.0             360.0             1.0   
5             4196.0       267.0             360.0             1.0   

  Property_Area  
1         Rural  
2         Urban  
3         Urban  
4         Urban  
5         Urban  


In [33]:
# Step 7: Create a search space for KNN
param_grid_knn = {'model__n_neighbors': list(range(1, 11))}

In [34]:
# Step 8: Fit a grid search with the pipeline for KNN
grid_search_knn = GridSearchCV(pipeline, param_grid_knn, cv=5)
grid_search_knn.fit(X_train, y_train)
best_model_knn = grid_search_knn.best_estimator_
y_pred_knn = best_model_knn.predict(X_test)
accuracy_grid_search_knn = accuracy_score(y_test, y_pred_knn)

In [35]:
# Step 9: Expand search space for Logistic Regression and Random Forest
param_grid = [
    {
        'model': [LogisticRegression()],
        'model__C': [0.001, 0.01, 0.1, 1, 10, 100]
    },
    {
        'model': [RandomForestClassifier()],
        'model__n_estimators': [10, 50, 100, 200]
    }
]

In [38]:
# Step 10: Fit a grid search with the expanded pipeline
grid_search = GridSearchCV(pipeline, param_grid, cv=5)
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)
accuracy_grid_search_best = accuracy_score(y_test, y_pred_best)

In [40]:
# Step 11: Summarize results
print("Default Model Accuracy (KNN):", accuracy_default)
print("Grid Search Best KNN Model Accuracy:", accuracy_grid_search_knn)
print("Grid Search Best Model:", best_model.named_steps['model'].__class__.__name__)
print("Grid Search Best Hyperparameters:", best_model.get_params()['model'])
print("Grid Search Best Model Accuracy:", accuracy_grid_search_best)

Default Model Accuracy (KNN): 0.78125
Grid Search Best KNN Model Accuracy: 0.7916666666666666
Grid Search Best Model: LogisticRegression
Grid Search Best Hyperparameters: LogisticRegression(C=10)
Grid Search Best Model Accuracy: 0.8229166666666666


In summary, the grid search with logistic regression as the best model and a regularization parameter (C) of 10 achieved the highest accuracy of approximately 82.29% on the test set, which is an improvement over the default KNN model's accuracy of 78.13%. The best model selected after grid search is a logistic regression model with C=10.